# Cross-validation RoBERTa-based model using Tree-parzen Estimator algorithm 

Install required libraries

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import RobertaTokenizer, TFRobertaModel
import matplotlib.pyplot as plt
from helpers import *
from implementations import *
import pickle
import os 
import json
from bson import json_util
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import KFold

To speed up the training and inferencing procedure we made use of the TPUs available on Google Colab. If you want to run this code on Google Colab, uncomment the cell below

In [4]:
# %tensorflow_version 2.x
# import tensorflow as tf
# print("Tensorflow version " + tf.__version__)

# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Tensorflow version 2.7.0
Running on TPU  ['10.4.54.218:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.4.54.218:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.4.54.218:8470


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Load the data (we use the smaller dataset due to running time) and set required constants

In [5]:
# Small dataset (200'000 observations)
POS_TRAINING_PATH = "train_pos.txt"
NEG_TRAINING_PATH = "train_neg.txt"

# Full dataset (2.2 million obsevations)
POS_TRAINING_PATH_FULL = "train_pos_full.txt"
NEG_TRAINING_PATH_FULL = "train_neg_full.txt"

# Test data
TEST_PATH = "test_data.txt"

# set the required constants
RANDOM_STATE = 123
MAX_LENGTH= 55
USE_LSTM = False # set to false to cross validate the model without Bi-LSTM layers 

# Load both small and large training sets
df_full = loadData(POS_TRAINING_PATH, NEG_TRAINING_PATH)
# df_full = loadData(POS_TRAINING_PATH_FULL, NEG_TRAINING_PATH_FULL)

Load the test data

In [6]:
# Load the test set.
test_sent = []
idx = []
with open(TEST_PATH) as test:
    for line in test:
        split = line.split(",", 1)
        idx.append(int(split[0]))
        test_sent.append(split[1])

data = {'index':idx,'tweet':test_sent}
df_test = pd.DataFrame(data)
df_test.head()

,index,tweet
0,1,sea doo pro sea scooter ( sports with the port...
1,2,<user> shucks well i work all week so now i ca...
2,3,i cant stay away from bug thats my baby\n
3,4,<user> no ma'am ! ! ! lol im perfectly fine an...
4,5,"whenever i fall asleep watching the tv , i alw..."


Load and initialize the base RoBERTa tokenizer

In [7]:
# RoBERTa
tokenizer_roberta = RobertaTokenizer.from_pretrained('roberta-base')

In [8]:
# we used the entire small dataset
training = df_full.sample(frac=1,random_state=123) 

In [10]:
training.head()

,tweet,sentiment
28285,"<user> awww , whizzy is happy\n",1
5080,#waystomakemehappy back rubs please\n,1
93291,this is torture ; i'm sat upstairs revising an...,0
3185,8am she's lucky she is cute for waking me so e...,1
61505,anyone who takes the mick out of anyone on #un...,1


Obtain the DataFrame of indeces and of attentions for each tweet. 

In [10]:
train_idx, train_att = batch_encode_cross_validation(tokenizer_roberta, training.tweet.to_list())

Define the function that builds the model

In [12]:
def build_model(transformer, dropout_rate, layer1_nodes, layer2_nodes, act_func, l1, l2, max_length=MAX_LENGTH, random_state=RANDOM_STATE):
    """""""""
    Build the model given the set of hyperparameters

    Input:
        - transformer:   the Transformer model to be used (either DistilBERT or RoBERTa)
        - dropout_rate:  dropout rate
        - layer1_nodes:  number of neurons in the frist layer of the MLP classifier
        - layer2_nodes:  number of neurons in the second layer of the MLP classifier
        - act_func:      activation function
        - l1:            l1 regularization weight
        - l2:            l2 regularization weight
        - max_length:    max sentence lenght (default=55) 
        - random_state:  seed for reproduceabilty
    Output:
        - model:         the built model ready to be compiled and trained
    """""""""

    # Define weight initializer with a random seed to ensure reproducibility
    weight_initializer = tf.keras.initializers.GlorotNormal(seed=random_state)

    # Define input layers
    input_ids_layer = tf.keras.layers.Input(shape=(max_length,),
                                            name='input_ids',
                                            dtype='int32')
    input_attention_layer = tf.keras.layers.Input(shape=(max_length,),
                                                  name='input_attention',
                                                  dtype='int32')

    # Access the last layer of the Transformer model.
    # It is a tf.Tensor of shape (batch_size, max_length, hidden_size=768).
    last_hidden_state = transformer([input_ids_layer, input_attention_layer])[0]

    # From the last layer, we select the embedding of the [CLS] token
    cls_token = last_hidden_state[:, 0, :]
    
        # include 2 bi-direction LSTM layers
    if USE_LSTM == True:
        # obtain the vector representation of each word (do not select the [CLS] and [SEP] special tokens)
        embeddings = last_hidden_state[:, 1:-1, :]
        X = tf.keras.layers.Dropout(dropout_rateseed=random_state)(embeddings)
        # Bi-LSTM layers 
        X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=1024, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2), kernel_initializer=weight_initializer, bias_initializer='zeros', return_sequences=True), merge_mode='ave')(X)        X = tf.keras.layers.Dropout(LAYER_DROPOUT)(X)
        X = tf.keras.layers.Dropout(dropout_rateseed=random_state)(X)
        X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=1024, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2), kernel_initializer=weight_initializer, bias_initializer='zeros', return_sequences=False), merge_mode='ave')(X)
        X = tf.concat([cls_token, X], axis=1)
    else:
        X = cls_token
        
    # Build the 2-layers MLP classifier
    X = tf.keras.layers.Dropout(dropout_rateseed=random_state)(X)
    X = tf.keras.layers.Dense(layer1_nodes, activation=act_func, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2), kernel_initializer=weight_initializer, bias_initializer='zeros')(X)
    X = tf.keras.layers.Dropout(dropout_rateseed=random_state)(X)
    if layer2_nodes != 0:
        X = tf.keras.layers.Dense(layer2_nodes, activation=act_func, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1, l2=l2), kernel_initializer=weight_initializer, bias_initializer='zeros')(X)
        X = tf.keras.layers.Dropout(dropout_rateseed=random_state)(X)

    # Define a single node that makes up the output layer (for binary classification)
    output = tf.keras.layers.Dense(1,
                                   activation='sigmoid',
                                   kernel_initializer=weight_initializer,
                                   kernel_constraint=None,
                                   bias_initializer='zeros'
                                   )(X)

    # Define the model
    model = tf.keras.Model([input_ids_layer, input_attention_layer], output)

    return model

Define the function that compiles and trains the model given the hyperparameters

In [13]:
def train_model(model, training_ids, training_attention, val_ids, val_attention, training_sentiment, val_sentiment, batch_size, lr, beta1, beta2, train_epochs=2, fine_tune_epochs=3):
    """""""""
    Compile and train the given model

    Input:
        - model:              the model built to be trained
        - training_ids:       training set of indeces
        - training_attention: training set of attention masks
        - val_ids:            validation set of indeces 
        - val_attention:      validation set of attention masks
        - training_sentiment: training labels
        - val_sentiment:      validation labels
        - batchsize:          batchsize to be used for training
        - lr:                 learning rate to be used for training
        - beta1:              beta_1 parameter of Adam optimizer
        - beta2:              beta_2 parameter of Adam optimizer
        - train_epochs:       number of epochs to train the full model (default=1)
        - fine_tune_epochs    number of epochs to fine tune the MLP classifier (default=3)
    Output:
        - val_acc:            best validation accuracy 
        - best_epoch:         fine_tune epochs that gave the best validation accuracy
    """""""""
    
    num_steps = tf.shape(training_ids)[0].numpy() // batch_size

    # compile the model using Adam optimizer
    model.compile(tf.keras.optimizers.Adam(lr=lr, beta_1=beta1, beta_2=beta2),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])

    print(model.summary())

    # Train the full model
    train_history1 = model.fit(
      x = [training_ids, training_attention],
      y = training_sentiment,
      epochs = train_epochs,
      batch_size = batch_size,
      steps_per_epoch = num_steps,
      validation_data = ([val_ids, val_attention], val_sentiment),
      verbose=2
    )

    # set the RoBERTa model to untrainable
    model.get_layer(index=2).trainable = False

    print(model.summary())

    # Fine tune the Neural Network built on top of the RoBERTa model
    train_history2 = model.fit(
      x = [training_ids, training_attention],
      y = training_sentiment,
      epochs = fine_tune_epochs,
      batch_size = batch_size,
      steps_per_epoch = num_steps,
      validation_data = ([val_ids, val_attention], val_sentiment),
      verbose=2
    )
    
    # Find the best validation accuracy
    val_acc = max(train_history2.history['val_accuracy'])
    print(val_acc)
    
    # Find the fine-tuning epoch that achieved the highest accuracy
    best_epoch = np.argmax(train_history2.history['val_accuracy']) + 1
    print(best_epoch)

    return val_acc, best_epoch


Define the function that will split the data into K-folds (we use 3 due to the runnning time) and trains and validates the current hyperparameters. Return the average accuracy over the folds

In [11]:
def hyperopt_cross_val(hyperparams):
    """""""""
    split the data into folds and run the training and validation on all the folds

    Input:
        - hyperparams:       set of hyperparameters to be used
    Output:
        - average accuracy:  average best valdiation accuracy 
        - best_epochs:       list of all the best epochs for each fold
    """""""""
    
    # uncomment the line below to use the TPU from Google Colab
    #with tpu_strategy.scope(): 

    kf = KFold(n_splits=3)
    accuracy = []
    best_epochs = []
    for train, test in kf.split(train_idx):
        # Initialize RoBERTa layer 
        RoBERTa = TFRobertaModel.from_pretrained('roberta-base')
        for layer in RoBERTa.layers:
            layer.trainable = True

        # Create datasets
        training_ids, training_attention = tf.convert_to_tensor(train_idx.iloc[train]), tf.convert_to_tensor(train_att.iloc[train])
        val_ids, val_attention = tf.convert_to_tensor(train_idx.iloc[test]), tf.convert_to_tensor(train_att.iloc[test])
        training_sentiment = training.sentiment.iloc[train]
        val_sentiment = training.sentiment.iloc[test]

        # Get hyperparameters
        (learning_rate, drop_rate, beta1, beta2, l1, l2, batchsize, activation, layer1_nodes, layer2_nodes) = hyperparams
        print("Training with the following hyperparameters:" )
        print(hyperparams)

        # Build model
        model = build_model(RoBERTa, drop_rate, layer1_nodes, layer2_nodes, activation, l1, l2, max_length=MAX_LENGTH, random_state=RANDOM_STATE)

        # compile model
        acc, epoch = train_model(model, training_ids, training_attention, val_ids, val_attention, training_sentiment, val_sentiment, batchsize, learning_rate, beta1, beta2)
        accuracy.append(acc)
        best_epochs.append(epoch)

    return np.mean(accuracy), best_epochs

Define the space of hyperparameters to be searched and run the Tree-Parzen Estimation algorithm for hyperparameter optimization 

In [ ]:
eval_num = 0
best_acc = None
best_hyperparams = None

# space to be searched 
parameter_space = [
                    hp.choice('learning_rate',[5e-5, 5e-4, 5e-3, 5e-6, 1e-5, 1e-4, 1e-3, 1e-6]),
                    hp.choice('drop_rate', [0.3, 0.4, 0.5, 0.6]),
                    hp.choice('beta1',    [0.9, 0.95, 0.97, 0.99]),
                    hp.choice('beta2',    [ 0.9, 0.95, 0.97, 0.99]),
                    hp.choice('l2',    [0.0001, 0.0005, 0.001, 0.005, 0.01, 0]),
                    hp.choice('l1',    [0.0001, 0.0005, 0.001, 0.005, 0.01, 0]),
                    hp.choice('batchsize',    [ 128, 256, 64]),
                    hp.choice('activation',   ['relu', 'tanh', 'sigmoid']),
                    hp.choice('layer1_nodes',  [1024, 512, 256]),
                    hp.choice('layer2_nodes',  [512, 256, 128, 64, 0]),
                  ]

def f(hyperparams):
    """""""""
    Function the TPE algorithm aims to minimize

    Input:
        - hyperparams:  set of hyperparameters to be used
    Output:
        - result:       dictionary of results 
    """""""""
    
    global eval_num
    global best_acc
    global best_hyperparams
    
    # run cross-validation
    acc, epochs = hyperopt_cross_val(hyperparams)
    print('FINISHED THE HYPERPARAMTER TUNING')
    print(acc)
    
    # save globally optimal accuracy
    if best_acc is None or acc > best_acc:
        best_acc = acc
        best_hyperparams = hyperparams
    
    # save results 
    result = {'acc': -acc, 'epochs': str(epochs),  'status': STATUS_OK, 'space': hyperparams}
    print("THE RESULTS ARE:")
    print(result)
    save_json_result(str(acc), result)
    return result

def run_a_trial():
    """""""""
    Function that runs one iteration of the TPE algorithm
    """""""""
    
    max_evals = nb_evals = 1

    print("Attempt to resume a past training if it exists:")
    try:
        # https://github.com/hyperopt/hyperopt/issues/267
        with open('results.pkl', 'rb') as in_file:
              trials = pickle.load(in_file)
        # trials = pickle.load(open("results.pkl", "rb"))
        print("Found saved Trials! Loading...")
        max_evals = len(trials.trials) + nb_evals
        print("Rerunning from {} trials to add another one.".format(len(trials.trials)))
        best = fmin(f, parameter_space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
    except:
        trials = Trials()
        print("Starting from scratch: nesw trials.")

    print("FINISHED FMIN")
    with open('results.pkl', 'wb') as out_file:
        pickle.dump(trials, out_file)
    print('best:')
    print(best)
      
while True:
    print("Optimizing New Model")
    try:
        run_a_trial()
    except Exception as err:
        err_str = str(err)
        print(err_str)


Optimizing New Model
Attempt to resume a past training if it exists:
Found saved Trials! Loading...
Rerunning from 0 trials to add another one.
  0%|          | 0/1 [00:00<?, ?it/s, best loss: ?]

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Training with the following hyperparameters:
(0.0001, 0.4, 0.9, 0.9, 0.01, 0.0005, 128, 'sigmoid', 512, 128)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 55)]         0           []                               
 input_attention (InputLayer)   [(None, 55)]         0           []                               
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 el)                            thPoolingAndCrossAt               'input_attention[0][0]']        
                                tentions(last_hidde                                               
                                n_state=(None, 55,                                                
                                768),                                               

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)



 bidirectional (Bidirectional)  (None, 53, 1024)     14688256    ['dropout_37[0][0]']             
 dropout_38 (Dropout)           (None, 53, 1024)     0           ['bidirectional[0][0]']          
 tf.__operators__.getitem (Slic  (None, 768)         0           ['tf_roberta_model[0][0]']       
 ingOpLambda)                                                                                     
 bidirectional_1 (Bidirectional  (None, 1024)        16785408    ['dropout_38[0][0]']             
 )                                                                                                
 tf.concat (TFOpLambda)         (None, 1792)         0           ['tf.__operators__.getitem[0][0]'
                                                                 , 'bidirectional_1[0][0]']       
 dropout_39 (Dropout)           (None, 1792)         0           ['tf.concat[0][0]']              
 dense (Dense)                  (None, 512)          918016      ['dropout_39[0][0]']             
 dropout_4

...

Training with the following hyperparameters:
(1e-05, 0.6, 0.9, 0.9, 0, 0.005, 64, 'tanh', 512, 256)
Model: "model_43"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 55)]         0           []                               
 input_attention (InputLayer)   [(None, 55)]         0           []                               
 tf_roberta_model_43 (TFRoberta  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 Model)                         thPoolingAndCrossAt               'input_attention[0][0]']        
                                tentions(last_hidde                                               
                                n_state=(None, 55,                                                
                                768),                                                     

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 55) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 55) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 1) dtype=int64>]


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 55) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 55) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 1) dtype=int64>]


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 55) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 55) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 1) dtype=int64>]


1888/1888 - 559s - loss: 20.2771 - accuracy: 0.8145 - val_loss: 9.5727 - val_accuracy: 0.8611 - 559s/epoch - 296ms/step

Model: "model_43"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 55)]         0           []                               
 input_attention (InputLayer)   [(None, 55)]         0           []                               
 tf_roberta_model_43 (TFRoberta  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 Model)                         thPoolingAndCrossAt               'input_attention[0][0]']        
                                tentions(last_hidde                                               
                                n_state=(None, 55,                                                
                                768),                                